In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext rpy2.ipython
%load_ext autotime

import sys
import os

nb_dir = os.path.split(os.getcwd())[0]

sys.path.append(nb_dir)
import pandas as pd

import numpy as np
import scipy as sp


from GELMMnet import GELMMnet
from sklearn.preprocessing import scale
from GELMMnet.utility.inference import _rmse
from GELMMnet.utility.kernels import kinship
from sklearn.metrics import mean_squared_error,r2_score

# Test kfold GridSearch

In [18]:
def test():
    import scipy as SP
    # data directory
    data_dir = './'
    
    # load genotypes
    geno_filename = os.path.join(data_dir,'genotypes.csv')
    X = SP.genfromtxt(geno_filename)
    [n_s,n_f] = X.shape

    # simulate phenotype
    SP.random.seed(1)
    n_c = 5
    idx = SP.random.randint(0,n_f,n_c)
    w = 1./n_c * SP.ones((n_c,1))
    ypheno = SP.dot(X[:,idx],w)
    ypheno = (ypheno-ypheno.mean())/ypheno.std()
    pheno_filename = os.path.join(data_dir,'phenotypes.csv')
    ypop = SP.genfromtxt(pheno_filename)
    ypop = SP.reshape(ypop,(n_s,1))
    y = 0.3*ypop + 0.5*ypheno + 0.2*SP.random.randn(n_s,1)
    y = (y-y.mean())/y.std()
    
    # init
    debug = False
    n_train = 150
    n_test = n_s - n_train
    n_reps = 100
    f_subset = 0.5
    mu = 10

    # split into training and testing
    train_idx = SP.random.permutation(SP.arange(n_s))
    test_idx = train_idx[n_train:]
    train_idx = train_idx[:n_train]

    # calculate kernel
    K = 1./n_f*SP.dot(X,X.T)
    

    
    import time
    print("Fit GELMMnet", )
    g = GELMMnet(y[train_idx],X[train_idx,:],K=K[train_idx][:,train_idx],intercept=False)
    start = time.time()
    g.kfoldFit(SP.identity(n_f),l1_nof=10, cpu=4, l2_nof=10, debug=True, max_iter = 100, eps = 1e-05)
    g.fit(SP.identity(n_f),l1=0.06833534, l2=1e-04, max_iter = 1000, eps = 1e-08)
    stop = time.time()
    print("CV took:", stop-start)
    return g
g = test()
g.post_selection_analysis(alpha=0.1, compute_intervals=True)

Fit GELMMnet
Correcting for population structure
Max l1: 0.341676704748
w != 0: []
L2: [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 0]
L1: [ 0.          0.03416767  0.06833534  0.10250301  0.13667068  0.17083835
  0.20500602  0.23917369  0.27334136  0.30750903]
Best Parameters: 0.03416767047478309 0.0 With error: -0.359500625658
Training Correlation: 0.9999896899454922  Training Error: 0.004730039978777899
sigma: -0.000762248974406
CV took: 9.877371072769165
Active [ 37  44  72 103 155 235 324 402 489 583 767 799 811 891 894 908]
(16, 150) (150,) (16,)


/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/utility/postselection.py:45: RuntimeWarning: invalid value encountered in double_scalars
  p = (norm.cdf(bb) - norm.cdf(zz)) / (norm.cdf(bb) - norm.cdf(aa))
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/utility/postselection.py:53: RuntimeWarning: overflow encountered in exp
  term1 = np.exp(zz * zz)
/Users/bs224/Dropbox/PostDoc/projects/GELMMnet/GELMMnet/utility/postselection.py:55: RuntimeWarning: overflow encountered in exp
  term1 = ff(aa) * np.exp(-(np.power(aa, 2) - np.power(zz, 2)) / 2)


,pval,coef0,beta,Zscore,lower_confidence,upper_confidence,lower_trunc,upper_trunc,sd
variable,,,,,,,,,
37.0,-0.0,0.339739,0.296605,-5185.038445,-0.006552,inf,0.0,0.0,-0.000066
44.0,-0.0,-0.051329,-0.002323,740.180452,-inf,0.006935,0.0,0.0,-0.000069
72.0,-0.0,0.330566,0.259747,-4797.409616,-0.006891,inf,0.0,0.0,-0.000069
103.0,-0.0,0.124316,0.024813,-1795.043558,-0.006926,inf,0.0,0.0,-0.000069
155.0,-0.0,0.105399,0.017799,-1513.284566,-0.006965,inf,0.0,0.0,-0.000070
235.0,-0.0,0.330020,0.303623,-5017.901832,-0.006577,inf,0.0,0.0,-0.000066
324.0,-0.0,-0.120964,-0.008135,1806.043767,-inf,0.006698,0.0,0.0,-0.000067
402.0,-0.0,0.024066,0.003623,-336.926292,-0.007143,inf,0.0,0.0,-0.000071
489.0,-0.0,0.123790,0.013044,-1803.956589,-0.006862,inf,0.0,0.0,-0.000069


time: 10.7 s


# Compare LMM-lasso

In [101]:
"""
lmm_lasso.py

Author:		Barbara Rakitsch
Year:		2012
Group:		Machine Learning and Computational Biology Group (http://webdav.tuebingen.mpg.de/u/karsten/group/)
Institutes:	Max Planck Institute for Developmental Biology and Max Planck Institute for Intelligent Systems (72076 Tuebingen, Germany)

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

import scipy as SP
import scipy.linalg as LA
import scipy.optimize as OPT
import pdb
import matplotlib.pylab as PLT
import time
import numpy as NP


def stability_selection(X,K,y,mu,n_reps,f_subset,**kwargs):
    """
    run stability selection

    Input:
    X: SNP matrix: n_s x n_f
    y: phenotype:  n_s x 1
    K: kinship matrix: n_s x n_s
    mu: l1-penalty

    n_reps:   number of repetitions
    f_subset: fraction of datasets that is used for creating one bootstrap

    output:
    selection frequency for all SNPs: n_f x 1
    """
    time_start = time.time()
    [n_s,n_f] = X.shape
    n_subsample = SP.ceil(f_subset * n_s)
    freq = SP.zeros(n_f)
    
    for i in range(n_reps):
        print('Iteration %d'%i)
        idx = SP.random.permutation(n_s)[:n_subsample]
        res = train(X[idx],K[idx][:,idx],y[idx],mu,**kwargs)
        snp_idx = (res['weights']!=0).flatten()
        freq[snp_idx] += 1.
        
    freq /= n_reps
    time_end = time.time()
    time_diff = time_end - time_start
    print('... finished in %.2fs'%(time_diff))
    return freq

def train(X,K,y,mu,numintervals=100,ldeltamin=-5,ldeltamax=5,rho=1,alpha=1,debug=False):
    """
    train linear mixed model lasso

    Input:
    X: SNP matrix: n_s x n_f
    y: phenotype:  n_s x 1
    K: kinship matrix: n_s x n_s
    mu: l1-penalty parameter
    numintervals: number of intervals for delta linesearch
    ldeltamin: minimal delta value (log-space)
    ldeltamax: maximal delta value (log-space)
    rho: augmented Lagrangian parameter for Lasso solver
    alpha: over-relatation parameter (typically ranges between 1.0 and 1.8) for Lasso solver

    Output:
    results
    """
    print('train LMM-Lasso')
    print('...l1-penalty: %.2f'%mu)
    
    time_start = time.time()
    [n_s,n_f] = X.shape
    assert X.shape[0]==y.shape[0], 'dimensions do not match'
    assert K.shape[0]==K.shape[1], 'dimensions do not match'
    assert K.shape[0]==X.shape[0], 'dimensions do not match'
    if y.ndim==1:
        y = SP.reshape(y,(n_s,1))

    # train null model
    S,U,ldelta0,monitor_nm = train_nullmodel(y,K,numintervals,ldeltamin,ldeltamax,debug=debug)
    print("ldelta", ldelta0)
    # train lasso on residuals
    delta0 = SP.exp(ldelta0)
    Sdi = 1./(S+delta0)
    Sdi_sqrt = SP.sqrt(Sdi)
    SUX = SP.dot(U.T,X)
    SUX = SUX * SP.tile(Sdi_sqrt,(n_f,1)).T
    SUy = SP.dot(U.T,y)
    SUy = SUy * SP.reshape(Sdi_sqrt,(n_s,1))
    
    w,monitor_lasso = train_lasso(SUX,SUy,mu,rho,alpha,debug=debug)

    time_end = time.time()
    time_diff = time_end - time_start
    print('... finished in %.2fs'%(time_diff))

    res = {}
    res['ldelta0'] = ldelta0
    res['weights'] = w
    res['time'] = time_diff
    res['monitor_lasso'] = monitor_lasso
    res['monitor_nm'] = monitor_nm
    return res


def predict(y_t,X_t,X_v,K_tt,K_vt,ldelta,w):
    """
    predict the phenotype

    Input:
    y_t: phenotype: n_train x 1
    X_t: SNP matrix: n_train x n_f
    X_v: SNP matrix: n_val x n_f
    K_tt: kinship matrix: n_train x n_train
    K_vt: kinship matrix: n_val  x n_train
    ldelta: kernel parameter
    w: lasso weights: n_f x 1

    Output:
    y_v: predicted phenotype: n_val x 1
    """
    print('predict LMM-Lasso')
    
    assert y_t.shape[0]==X_t.shape[0], 'dimensions do not match'
    assert y_t.shape[0]==K_tt.shape[0], 'dimensions do not match'
    assert y_t.shape[0]==K_tt.shape[1], 'dimensions do not match'
    assert y_t.shape[0]==K_vt.shape[1], 'dimensions do not match'
    assert X_v.shape[0]==K_vt.shape[0], 'dimensions do not match'
    assert X_t.shape[1]==X_v.shape[1], 'dimensions do not match'
    assert X_t.shape[1]==w.shape[0], 'dimensions do not match'
    
    [n_train,n_f] = X_t.shape
    n_test = X_v.shape[0]
    
    if y_t.ndim==1:
        y_t = SP.reshape(y_t,(n_train,1))
    if w.ndim==1:
        w = SP.reshape(w,(n_f,1))
    
    delta = SP.exp(ldelta)
    idx = w.nonzero()[0]

    if idx.shape[0]==0:
        return SP.dot(K_vt,LA.solve(K_tt + delta*SP.eye(n_train),y_t))
        
    y_v = SP.dot(X_v[:,idx],w[idx]) + SP.dot(K_vt, LA.solve(K_tt + delta*SP.eye(n_train),y_t-SP.dot(X_t[:,idx],w[idx])))
    return y_v

"""
helper functions
"""

def train_lasso(X,y,mu,rho=1,alpha=1,max_iter=5000,abstol=1E-4,reltol=1E-2,zero_threshold=1E-3,debug=False):
    """
    train lasso via Alternating Direction Method of Multipliers:
    min_w  0.5*sum((y-Xw)**2) + mu*|z|
    
    Input:
    X: design matrix: n_s x n_f
    y: outcome:  n_s x 1
    mu: l1-penalty parameter
    rho: augmented Lagrangian parameter
    alpha: over-relatation parameter (typically ranges between 1.0 and 1.8)

    the implementation is a python version of Boyd's matlab implementation of ADMM-Lasso, which can be found at:
    http://www.stanford.edu/~boyd/papers/admm/lasso/lasso.html

    more information about ADMM can be found in the paper linked at:
    http://www.stanford.edu/~boyd/papers/distr_opt_stat_learning_admm.html

    In particular, you can use any other Lasso-Solver instead. For the experiments, reported in the paper,
    we used the l1-solver from the package scikits. We didn't apply it here to avoid third-party packages.
    """
    if debug:
        print('... train lasso')

    # init
    [n_s,n_f] = X.shape
    w = SP.zeros((n_f,1))
    z = SP.zeros((n_f,1))
    u = SP.zeros((n_f,1))

    monitor = {}
    monitor['objval'] = []
    monitor['r_norm'] = []
    monitor['s_norm'] = []
    monitor['eps_pri'] = []
    monitor['eps_dual'] = []

    # cache factorization
    U = factor(X,rho)

    # save a matrix-vector multiply
    Xy = SP.dot(X.T,y)

    if debug:
        print('i\tobj\t\tr_norm\t\ts_norm\t\teps_pri\t\teps_dual')

    for i in range(max_iter):
        # w-update
        q = Xy + rho*(z-u)
        w = q/rho - SP.dot(X.T,LA.cho_solve((U,False),SP.dot(X,q)))/rho**2

        # z-update with relaxation
        zold = z
        w_hat = alpha*w + (1-alpha)*zold
        z = soft_thresholding(w_hat+u, mu/rho)

        # u-update
        u = u + (w_hat - z)

        monitor['objval'].append(lasso_obj(X,y,w,mu,z))
        monitor['r_norm'].append(LA.norm(w-z))
        monitor['s_norm'].append(LA.norm(rho*(z-zold)))
        monitor['eps_pri'].append(SP.sqrt(n_f)*abstol + reltol*max(LA.norm(w),LA.norm(z)))
        monitor['eps_dual'].append(SP.sqrt(n_f)*abstol + reltol*LA.norm(rho*u))

        if debug:
            print('%3d\t%10.4f\t%10.4f\t%10.4f\t%10.4f\t%10.2f'%(i,monitor['objval'][i],monitor['r_norm'][i],
                                                                 monitor['s_norm'][i],monitor['eps_pri'][i],monitor['eps_dual'][i]))

        if monitor['r_norm'][i]<monitor['eps_pri'][i] and monitor['r_norm'][i]<monitor['eps_dual'][i]:
            break

    
    w[SP.absolute(w)<zero_threshold]=0
    return w,monitor


def nLLeval(ldelta,Uy,S):
    """
    evaluate the negative log likelihood of a random effects model:
    nLL = 1/2(n_s*log(2pi) + logdet(K) + 1/ss * y^T(K + deltaI)^{-1}y,
    where K = USU^T.

    Uy: transformed outcome: n_s x 1
    S:  eigenvectors of K: n_s
    ldelta: log-transformed ratio sigma_gg/sigma_ee
    """
    n_s = Uy.shape[0]
    delta = SP.exp(ldelta)
    
    # evaluate log determinant
    Sd = S+delta
    ldet = SP.sum(SP.log(Sd))

    # evaluate the variance    
    Sdi = 1.0/Sd
    Uy = Uy.flatten()
    ss = 1./n_s * (Uy*Uy*Sdi).sum()

    # evalue the negative log likelihood
    nLL=0.5*(n_s*SP.log(2.0*SP.pi)+ldet+n_s+n_s*SP.log(ss));

    return nLL


def train_nullmodel(y,K,numintervals=100,ldeltamin=-5,ldeltamax=5,debug=False):
    """
    train random effects model:
    min_{delta}  1/2(n_s*log(2pi) + logdet(K) + 1/ss * y^T(K + deltaI)^{-1}y,
    
    Input:
    X: SNP matrix: n_s x n_f
    y: phenotype:  n_s x 1
    K: kinship matrix: n_s x n_s
    mu: l1-penalty parameter
    numintervals: number of intervals for delta linesearch
    ldeltamin: minimal delta value (log-space)
    ldeltamax: maximal delta value (log-space)
    """
    if debug:
        print('... train null model')
        
    n_s = y.shape[0]

    # rotate data
    S,U = LA.eigh(K)
    Uy = SP.dot(U.T,y)

    # grid search
    nllgrid=SP.ones(numintervals+1)*SP.inf
    ldeltagrid=SP.arange(numintervals+1)/(numintervals*1.0)*(ldeltamax-ldeltamin)+ldeltamin
    nllmin=SP.inf
    for i in SP.arange(numintervals+1):
        nllgrid[i]=nLLeval(ldeltagrid[i],Uy,S);
        
    # find minimum
    nll_min = nllgrid.min()
    ldeltaopt_glob = ldeltagrid[nllgrid.argmin()]

    # more accurate search around the minimum of the grid search
    for i in SP.arange(numintervals-1)+1:
        if (nllgrid[i]<nllgrid[i-1] and nllgrid[i]<nllgrid[i+1]):
            ldeltaopt,nllopt,iter,funcalls = OPT.brent(nLLeval,(Uy,S),(ldeltagrid[i-1],ldeltagrid[i],ldeltagrid[i+1]),full_output=True);
            if nllopt<nllmin:
                nllmin=nllopt;
                ldeltaopt_glob=ldeltaopt;

    monitor = {}
    monitor['nllgrid'] = nllgrid
    monitor['ldeltagrid'] = ldeltagrid
    monitor['ldeltaopt'] = ldeltaopt_glob
    monitor['nllopt'] = nllmin
    
    return S,U,ldeltaopt_glob,monitor
 

def factor(X,rho):
    """
    computes cholesky factorization of the kernel K = 1/rho*XX^T + I

    Input:
    X design matrix: n_s x n_f (we assume n_s << n_f)
    rho: regularizaer

    Output:
    L  lower triangular matrix
    U  upper triangular matrix
    """
    n_s,n_f = X.shape
    K = 1/rho*SP.dot(X,X.T) + SP.eye(n_s)
    U = LA.cholesky(K)
    return U


def soft_thresholding(w,kappa):
    """
    Performs elementwise soft thresholding for each entry w_i of the vector w:
    s_i= argmin_{s_i}  rho*abs(s_i) + rho/2*(x_i-s_i) **2
    by using subdifferential calculus

    Input:
    w vector nx1
    kappa regularizer

    Output:
    s vector nx1
    """
    n_f = w.shape[0]
    zeros = SP.zeros((n_f,1))
    s = NP.max(SP.hstack((w-kappa,zeros)),axis=1) - NP.max(SP.hstack((-w-kappa,zeros)),axis=1)
    s = SP.reshape(s,(n_f,1))
    return s


def lasso_obj(X,y,w,mu,z):
    """
    evaluates lasso objective: 0.5*sum((y-Xw)**2) + mu*|z|

    Input:
    X: design matrix: n_s x n_f
    y: outcome:  n_s x 1
    mu: l1-penalty parameter
    w: weights: n_f x 1
    z: slack variables: n_fx1

    Output:
    obj
    """
    return 0.5*((SP.dot(X,w)-y)**2).sum() + mu*SP.absolute(z).sum()



    # train null model
    S,U,ldelta0,monitor_nm = train_nullmodel(y,K,numintervals,ldeltamin,ldeltamax,debug=debug)
    print("ldelta", ldelt0)
    # train lasso on residuals
    delta0 = SP.exp(ldelta0)
    Sdi = 1./(S+delta0)
    Sdi_sqrt = SP.sqrt(Sdi)
    SUX = SP.dot(U.T,X)
    SUX = SUX * SP.tile(Sdi_sqrt,(n_f,1)).T
    SUy = SP.dot(U.T,y)
    SUy = SUy * SP.reshape(Sdi_sqrt,(n_s,1))
    
    w,monitor_lasso = train_lasso(SUX,SUy,mu,rho,alpha,debug=debug)

    time_end = time.time()
    time_diff = time_end - time_start
    print('... finished in %.2fs'%(time_diff))

    res = {}
    res['ldelta0'] = ldelta0
    res['weights'] = w
    res['time'] = time_diff
    res['monitor_lasso'] = monitor_lasso
    res['monitor_nm'] = monitor_nm
    return res

def test():
    # data directory
    data_dir = './'
    
    # load genotypes
    geno_filename = os.path.join(data_dir,'genotypes.csv')
    X = SP.genfromtxt(geno_filename)
    [n_s,n_f] = X.shape

    # simulate phenotype
    SP.random.seed(1)
    n_c = 5
    idx = SP.random.randint(0,n_f,n_c)
    w = 1./n_c * SP.ones((n_c,1))
    ypheno = SP.dot(X[:,idx],w)
    ypheno = (ypheno-ypheno.mean())/ypheno.std()
    pheno_filename = os.path.join(data_dir,'phenotypes.csv')
    ypop = SP.genfromtxt(pheno_filename)
    ypop = SP.reshape(ypop,(n_s,1))
    y = 0.3*ypop + 0.5*ypheno + 0.2*SP.random.randn(n_s,1)
    y = (y-y.mean())/y.std()
    
    # init
    debug = False
    n_train = 150
    n_test = n_s - n_train
    n_reps = 100
    f_subset = 0.5
    mu = 10

    # split into training and testing
    train_idx = SP.random.permutation(SP.arange(n_s))
    test_idx = train_idx[n_train:]
    train_idx = train_idx[:n_train]

    # calculate kernel
    K = 1./n_f*SP.dot(X,X.T)
    
    # train
    res = train(X[train_idx],K[train_idx][:,train_idx],y[train_idx],mu,debug=debug)
    w = res['weights']
    print('... number of Nonzero Weights: %d'%(w!=0).sum())
    ldelta0 = res['ldelta0']
    
    # predict
    yhat = predict(y[train_idx],X[train_idx,:],X[test_idx,:],K[train_idx][:,train_idx],K[test_idx][:,train_idx],ldelta0,w)
    #print(yhat)
    w1=w[:,0]
    print("w",w1[w1 != 0])
    print("r2:", r2_score(y[test_idx], yhat))
    corr = 1./n_test * ((yhat-yhat.mean())*(y[test_idx]-y[test_idx].mean())).sum()/(yhat.std()*y[test_idx].std())
    print('... corr(Yhat,Ytrue): %.2f (in percent)'%(corr))
    print()
    
    import time
    print("Fit GELMMnet", )
    g = GELMMnet(y[train_idx],X[train_idx,:],K=K[train_idx][:,train_idx],intercept=False)
    start = time.time()
    g.kfoldFit(SP.identity(n_f),l1_nof=100, cpu=1, l2_nof=10, debug=False, max_iter = 100, eps = 1e-05)
    #g.fit(SP.identity(n_f),l1=0.06833534, l2=1e-04, max_iter = 1000, eps = 1e-08)
    stop = time.time()
    SUX = g.SUX
    SUy = g.SUy
    #g.w=w1
    #g.b = 0
    print("CV took:", stop-start)
    yhat2 = g.predict(X[test_idx,:])
    yhat2 = SP.reshape(yhat2, (len(yhat2), 1))
    #print("w",g.w[g.w != 0])
    print("b",g.b)
    #print(yhat)
    print('... number of Nonzero Weights: %d'%(g.w !=0).sum())
    corr = 1./n_test * ((yhat2-yhat2.mean())*(y[test_idx,0]-y[test_idx,0].mean())).sum()/(yhat2.std()*y[test_idx,0].std())
    print("r2:", r2_score(y[test_idx,0], yhat2))
    print('... corr(Yhat,Ytrue): %.2f (in percent)'%(corr))
    print()
    return g
g=test()
g.post_selection_analysis(alpha=0.1,compute_intervals=True)

train LMM-Lasso
...l1-penalty: 10.00
ldelta -0.988297084887
... finished in 0.81s
... number of Nonzero Weights: 14
predict LMM-Lasso
w [ 0.29842933 -0.00451417  0.26317048  0.02745376  0.01940525  0.30468686
 -0.00977561  0.0036733   0.01539739  0.07248216  0.23095535 -0.01170971
 -0.02254671  0.27645177]
r2: 0.785183005721
... corr(Yhat,Ytrue): 0.89 (in percent)

Fit GELMMnet
CV took: 53.429972887039185
b 0.0
... number of Nonzero Weights: 549
r2: 0.266598871538
... corr(Yhat,Ytrue): 0.00 (in percent)



,pval,coef,beta,Zscore,lower_confidence,upper_confidence,lower_trunc,upper_trunc,sd
variable,,,,,,,,,
1.0,1.0,0.011674,0.028086,-12335.289481,-inf,0.000095,0.0,0.0,-9.463800e-07
2.0,1.0,-0.034393,-0.118159,64124.009857,-0.000054,inf,0.0,0.0,-5.363446e-07
3.0,1.0,0.034757,0.085960,-82055.073221,-inf,0.000042,0.0,0.0,-4.235769e-07
4.0,1.0,0.001940,0.018810,-3939.396113,-inf,0.000049,0.0,0.0,-4.923758e-07
5.0,1.0,-0.004572,0.000557,10760.064376,-0.000042,inf,0.0,0.0,-4.248675e-07
6.0,1.0,-0.041264,-0.137374,87690.552569,-0.000047,inf,0.0,0.0,-4.705592e-07
8.0,1.0,0.048214,0.159356,-102985.220268,-inf,0.000047,0.0,0.0,-4.681615e-07
9.0,1.0,0.019240,0.078472,-46137.613811,-inf,0.000042,0.0,0.0,-4.170182e-07
10.0,1.0,-0.003261,-0.000858,8571.803583,-0.000038,inf,0.0,0.0,-3.804246e-07


time: 1min 13s


In [64]:
%%R -i y,X,SUX,SUy
library("gelnet")

#w=gelnet.cv(SUX, SUy[,1], 10, 10)
#w
nL2=10
v1 <- 1:(as.integer( nL2 / 2 ))
if( nL2 %% 2 == 0 )
      vL2 <- 10 ^ c(rev(1-v1), v1)
else
      vL2 <- 10 ^ c(rev(-v1), 0, v1)
vL2

nL1 = 10
L1s = 0.341676704748
seq( 0, L1s, length.out = nL1+1 )[1:nL1]

 [1] 0.00000000 0.03416767 0.06833534 0.10250301 0.13667068 0.17083835
 [7] 0.20500602 0.23917369 0.27334136 0.30750903


time: 64 ms


In [61]:
import numpy as np
nL2 = 10
n = max(1,int(nL2/ 2))
print(n)
[10**x for x in range(-n,n)]+[0]


5


[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 0]

time: 4.54 ms


# Comparing with selectiveInference

In [56]:
%%R -o x,y,out
library(glmnet)
#library(selectiveInference)
set.seed(43)
n = 50
p = 10
sigma = 1
x = matrix(rnorm(n*p),n,p)
x=scale(x,TRUE,TRUE)
beta = c(3,2,rep(0,p-2))
y = x%*%beta + sigma*rnorm(n)
# first run glmnet
#lambdahat = cv.glmnet(x,y,standardize=FALSE)
#print(lambdahat)
#sigmahat = estimateSigma(x,y, standardize=FALSE, intercept=F)$sigmahat

gfit = glmnet(x,y,standardize=FALSE)
# extract coef for a given lambda; note the 1/n factor!
# (and we don't save the intercept term) fixedLassoInf 7 
lambda = .8
args = coef(gfit, s=lambda/n, exact=TRUE)
print(args)
# compute fixed lambda p-values and selection intervals
out = fixedLassoInf(x,y,args[-1],lambda,sigma=sigma)
out


11 x 1 sparse Matrix of class "dgCMatrix"
                      1
(Intercept)  0.05496141
V1           2.90117931
V2           2.09140871
V3           0.10536154
V4           0.07723765
V5          -0.17150755
V6          -0.13135784
V7           0.13166391
V8           0.02265503
V9          -0.09731043
V10          .         
[1] "vec"
               [,1]
 [1,]    0.06785655
 [2,] -163.10265951
 [3,]   21.50032916
 [4,]   27.09518743
 [5,]   25.72685810
 [6,]   10.14068596
 [7,]   23.52928281
 [8,]   21.15154523
 [9,]   13.44107098
[1]  9 50
NULL
[1] 50  1
[1] 13.44107
[1] 21.15155
[1] "vec"
              [,1]
 [1,] -238.1489329
 [2,]    0.2020505
 [3,]    9.6753307
 [4,]  -47.6552630
 [5,]   10.3708646
 [6,]  -52.7019431
 [7,]   -5.2940346
 [8,]   11.0129242
 [9,]   28.6477869
[1]  9 50
NULL
[1] 50  1
[1] 11.01292
[1] 28.64779
[1] "vec"
             [,1]
 [1,]  81.0509366
 [2,] -91.0714810
 [3,]   0.2055169
 [4,]  -0.9086174
 [5,]  -3.4977034
 [6,]   4.8123953
 [7,]   9.2039893
 [8,

time: 45.9 ms


In [63]:
from GELMMnet import GELMMnet

P = np.identity(x.shape[1])
g = GELMMnet(y, x)
g.fit(P,l1=0.8/x.shape[0], l2=0 )
g.l1 = 0.8
g.sigma = 1
l1 = g.l1
print("l1",l1)
g.post_selection_analysis(alpha=0.1, compute_intervals=True)

l1 0.8


,pval,coef,beta,Zscore,lower_confidence,upper_confidence,lower_trunc,upper_trunc,sd
variable,,,,,,,,,
0.0,3.496488e-39,2.911576,2.901111,18.896700,2.655225,3.196017,0.048041,0.049253,0.154079
1.0,7.364081e-16,2.122611,2.091549,13.803195,1.856116,2.377021,0.048605,0.049018,0.153777
2.0,4.947668e-01,0.141078,0.105587,0.816518,-0.728072,0.403701,0.049453,0.048846,0.172780
3.0,5.783408e-01,0.109437,0.077349,0.699300,-0.854250,0.336910,0.049754,0.048564,0.156495
4.0,2.566885e-01,-0.205650,-0.171590,-1.240091,-0.485609,0.309639,0.048488,0.049419,0.165835
5.0,3.584677e-01,-0.144647,-0.131332,-0.967806,-0.394957,0.391907,0.048450,0.049335,0.149458
6.0,3.739127e-01,0.141072,0.131656,0.908924,-0.432321,0.476662,0.049783,0.048949,0.155208
7.0,8.650898e-01,0.045156,0.022705,0.303971,-2.872273,0.165213,0.049930,0.049556,0.148555
8.0,5.103043e-01,-0.112833,-0.097332,-0.722396,-0.355381,0.658173,0.049782,0.049472,0.156193


time: 899 ms


# Work in progress

In [55]:
%%R 

# Special linear time order function, works only when x
# is a scrambled vector of integers.

Order <- function(x) {
  n = length(x)
  o = numeric(n)
  o[x] = Seq(1,n)
  return(o)
}

# Returns a sequence of integers from a to b if a <= b,
# otherwise nothing. You have no idea how important this
# function is...

Seq <- function(a, b, ...) {
  if (a<=b) return(seq(a,b,...))
  else return(numeric(0))
}

# Returns the sign of x, with Sign(0)=1.

Sign <- function(x) {
  return(-1+2*(x>=0))
}

##############################

# Centering and scaling convenience function

standardize <- function(x, y, intercept, normalize) {
  x = as.matrix(x)
  y = as.numeric(y)
  n = nrow(x)
  p = ncol(x)

  if (intercept) {
    bx = colMeans(x)
    by = mean(y)
    x = scale(x,bx,FALSE)
    y = y-mean(y)
  } else {
    bx = rep(0,p)
    by = 0
  }
  if (normalize) {
    sx = sqrt(colSums(x^2))
    x = scale(x,FALSE,sx)
  } else {
    sx = rep(1,p)
  }

  return(list(x=x,y=y,bx=bx,by=by,sx=sx))
}

##############################

# Interpolation function to get coefficients

coef.interpolate <- function(beta, s, knots, decreasing=TRUE) {
  # Sort the s values
  o = order(s,decreasing=decreasing)
  s = s[o]

  k = length(s)
  mat = matrix(rep(knots,each=k),nrow=k)
  if (decreasing) b = s >= mat
  else b = s <= mat
  blo = max.col(b,ties.method="first")
  bhi = pmax(blo-1,1)

  i = bhi==blo
  p = numeric(k)
  p[i] = 0
  p[!i] = ((s-knots[blo])/(knots[bhi]-knots[blo]))[!i]

  beta = t((1-p)*t(beta[,blo,drop=FALSE]) + p*t(beta[,bhi,drop=FALSE]))
  colnames(beta) = as.character(round(s,3))
  rownames(beta) = NULL

  # Return in original order
  o = order(o)
  return(beta[,o,drop=FALSE])
}

##############################

checkargs.xy <- function(x, y) {
  if (missing(x)) stop("x is missing")
  if (is.null(x) || !is.matrix(x)) stop("x must be a matrix")
  if (missing(y)) stop("y is missing")
  if (is.null(y) || !is.numeric(y)) stop("y must be numeric")
  if (ncol(x) == 0) stop("There must be at least one predictor [must have ncol(x) > 0]")
  if (checkcols(x)) stop("x cannot have duplicate columns")
  if (length(y) == 0) stop("There must be at least one data point [must have length(y) > 0]")
  if (length(y)!=nrow(x)) stop("Dimensions don't match [length(y) != nrow(x)]")
}

checkargs.misc <- function(sigma=NULL, alpha=NULL, k=NULL,
                           gridrange=NULL, gridpts=NULL, griddepth=NULL,
                           mult=NULL, ntimes=NULL,
                           beta=NULL, lambda=NULL, tol.beta=NULL, tol.kkt=NULL,
                           bh.q=NULL) {

  if (!is.null(sigma) && sigma <= 0) stop("sigma must be > 0")
  if (!is.null(lambda) && lambda < 0) stop("lambda must be >= 0")
  if (!is.null(alpha) && (alpha <= 0 || alpha >= 1)) stop("alpha must be between 0 and 1")
  if (!is.null(k) && length(k) != 1) stop("k must be a single number")
  if (!is.null(k) && (k < 1 || k != floor(k))) stop("k must be an integer >= 1")
  if (!is.null(gridrange) && (length(gridrange) != 2 || gridrange[1] > gridrange[2]))
    stop("gridrange must be an interval of the form c(a,b) with a <= b")
  if (!is.null(gridpts) && (gridpts < 20 || gridpts != round(gridpts)))
    stop("gridpts must be an integer >= 20")
  if (!is.null(griddepth) && (griddepth > 10 || griddepth != round(griddepth)))
    stop("griddepth must be an integer <= 10")
  if (!is.null(mult) && mult < 0) stop("mult must be >= 0")
  if (!is.null(ntimes) && (ntimes <= 0 || ntimes != round(ntimes)))
    stop("ntimes must be an integer > 0")
  if (!is.null(beta) && sum(beta!=0)==0) stop("Value of lambda too large, beta is zero")
  if (!is.null(lambda) && length(lambda) != 1) stop("lambda must be a single number")
  if (!is.null(lambda) && lambda < 0) stop("lambda must be >=0")
  if (!is.null(tol.beta) && tol.beta <= 0) stop("tol.beta must be > 0")
  if (!is.null(tol.kkt) && tol.kkt <= 0) stop("tol.kkt must be > 0")
}

# Make sure that no two columms of A are the same
# (this works with probability one).

checkcols <- function(A) {
  b = rnorm(nrow(A))
  a = sort(t(A)%*%b)
  if (any(diff(a)==0)) return(TRUE)
  return(FALSE)
}

estimateSigma <- function(x, y, intercept=TRUE, standardize=TRUE) {
  checkargs.xy(x,rep(0,nrow(x)))
  if(nrow(x)<10) stop("Number of observations must be at least 10 to run estimateSigma")
  cvfit=cv.glmnet(x,y,intercept=intercept,standardize=standardize)
  lamhat=cvfit$lambda.min
  fit=glmnet(x,y,standardize=standardize)
  yhat=predict(fit,x,s=lamhat)
  nz=sum(predict(fit,s=lamhat, type="coef")!=0)
  sigma=sqrt(sum((y-yhat)^2)/(length(y)-nz-1))
  return(list(sigmahat=sigma, df=nz))
}

# Update the QR factorization, after a column has been
# added. Here Q1 is m x n, Q2 is m x k, and R is n x n.

updateQR <- function(Q1,Q2,R,col) {
  m = nrow(Q1)
  n = ncol(Q1)
  k = ncol(Q2)

  a = .C("update1",
    Q2=as.double(Q2),
    w=as.double(t(Q2)%*%col),
    m=as.integer(m),
    k=as.integer(k),
    dup=FALSE,
    package="selectiveInference")

  Q2 = matrix(a$Q2,nrow=m)
  w = c(t(Q1)%*%col,a$w)

  # Re-structure: delete a column from Q2, add one to
  # Q1, and expand R
  Q1 = cbind(Q1,Q2[,1])
  Q2 = Q2[,-1,drop=FALSE]
  R = rbind(R,rep(0,n))
  R = cbind(R,w[Seq(1,n+1)])

  return(list(Q1=Q1,Q2=Q2,R=R))
}

# Moore-Penrose pseudo inverse for symmetric matrices

pinv <- function(A, tol=.Machine$double.eps) {
  e = eigen(A)
  v = Re(e$vec)
  d = Re(e$val)
  d[d > tol] = 1/d[d > tol]
  d[d < tol] = 0
  if (length(d)==1) return(v*d*v)
  else return(v %*% diag(d) %*% t(v))
}



# Assuming that grid is in sorted order from smallest to largest,
# and vals are monotonically increasing function values over the
# grid, returns the grid end points such that the corresponding
# vals are approximately equal to {val1, val2}

grid.search <- function(grid, fun, val1, val2, gridpts=100, griddepth=2) {
  n = length(grid)
  vals = fun(grid)
    
  ii = which(vals >= val1)
  jj = which(vals <= val2)
  if (length(ii)==0) return(c(grid[n],Inf))   # All vals < val1
  if (length(jj)==0) return(c(-Inf,grid[1]))  # All vals > val2
  # RJT: the above logic is correct ... but for simplicity, instead,
  # we could just return c(-Inf,Inf) 

  i1 = min(ii); i2 = max(jj)
  if (i1==1) lo = -Inf
  else lo = grid.bsearch(grid[i1-1],grid[i1],fun,val1,gridpts,
         griddepth-1,below=TRUE)
  if (i2==n) hi = Inf
  else hi = grid.bsearch(grid[i2],grid[i2+1],fun,val2,gridpts,
         griddepth-1,below=FALSE)
  return(c(lo,hi))
}

# Repeated bin search to find the point x in the interval [left, right]
# that satisfies f(x) approx equal to val. If below=TRUE, then we seek
# x such that the above holds and f(x) <= val; else we seek f(x) >= val.

grid.bsearch <- function(left, right, fun, val, gridpts=100, griddepth=1, below=TRUE) {
  n = gridpts
  depth = 1

  while (depth <= griddepth) {
    grid = seq(left,right,length=n)
    vals = fun(grid)
    
    if (below) {
      ii = which(vals >= val)
      if (length(ii)==0) return(grid[n])   # All vals < val (shouldn't happen)
      if ((i0=min(ii))==1) return(grid[1]) # All vals > val (shouldn't happen)
      left = grid[i0-1]
      right = grid[i0]
    }
    
    else {
      ii = which(vals <= val)
      if (length(ii)==0) return(grid[1])   # All vals > val (shouldn't happen)
      if ((i0=max(ii))==n) return(grid[n]) # All vals < val (shouldn't happen)
      left = grid[i0]
      right = grid[i0+1]
    }

    depth = depth+1
  }

  return(ifelse(below, left, right))
}

# Returns Prob(Z>z | Z in [a,b]), where mean can be a vector

tnorm.surv <- function(z, mean, sd, a, b, bits=NULL) {
  z = max(min(z,b),a)
  
  # Check silly boundary cases
  p = numeric(length(mean))
  p[mean==-Inf] = 0
  p[mean==Inf] = 1
  
  # Try the multi precision floating point calculation first
  o = is.finite(mean)
  mm = mean[o]
  pp = mpfr.tnorm.surv(z,mm,sd,a,b,bits) 

  # If there are any NAs, then settle for an approximation
  oo = is.na(pp)
  if (any(oo)) pp[oo] = bryc.tnorm.surv(z,mm[oo],sd,a,b)
  
  p[o] = pp
  return(p)
}

# Returns Prob(Z>z | Z in [a,b]), where mean cane be a vector, using
# multi precision floating point calculations thanks to the Rmpfr package

mpfr.tnorm.surv <- function(z, mean=0, sd=1, a, b, bits=NULL) {
  # If bits is not NULL, then we are supposed to be using Rmpf
  # (note that this was fail if Rmpfr is not installed; but
  # by the time this function is being executed, this should
  # have been properly checked at a higher level; and if Rmpfr
  # is not installed, bits would have been previously set to NULL)
  if (!is.null(bits)) {
    z = Rmpfr::mpfr((z-mean)/sd, precBits=bits)
    a = Rmpfr::mpfr((a-mean)/sd, precBits=bits)
    b = Rmpfr::mpfr((b-mean)/sd, precBits=bits)
    return(as.numeric((Rmpfr::pnorm(b)-Rmpfr::pnorm(z))/
                      (Rmpfr::pnorm(b)-Rmpfr::pnorm(a))))
  }
  
  # Else, just use standard floating point calculations
  z = (z-mean)/sd
  a = (a-mean)/sd
  b = (b-mean)/sd
  return((pnorm(b)-pnorm(z))/(pnorm(b)-pnorm(a)))
}

# Returns Prob(Z>z | Z in [a,b]), where mean can be a vector, based on
# A UNIFORM APPROXIMATION TO THE RIGHT NORMAL TAIL INTEGRAL, W Bryc
# Applied Mathematics and Computation
# Volume 127, Issues 23, 15 April 2002, Pages 365--374
# https://math.uc.edu/~brycw/preprint/z-tail/z-tail.pdf

bryc.tnorm.surv <- function(z, mean=0, sd=1, a, b) {
  z = (z-mean)/sd
  a = (a-mean)/sd
  b = (b-mean)/sd
  n = length(mean)

  term1 = exp(z*z)
  o = a > -Inf
  term1[o] = ff(a[o])*exp(-(a[o]^2-z[o]^2)/2)
  term2 = rep(0,n)
  oo = b < Inf
  term2[oo] = ff(b[oo])*exp(-(b[oo]^2-z[oo]^2)/2)
  p = (ff(z)-term2)/(term1-term2)

  # Sometimes the approximation can give wacky p-values,
  # outside of [0,1] ..
  #p[p<0 | p>1] = NA
  p = pmin(1,pmax(0,p))
  return(p)
}

ff <- function(z) {
  return((z^2+5.575192695*z+12.7743632)/
         (z^3*sqrt(2*pi)+14.38718147*z*z+31.53531977*z+2*12.77436324))
}

# Return Prob(Z>z | Z in [a,b]), where mean can be a vector, based on
# Riemann approximation tricks, by Max G'Sell

gsell.tnorm.surv <- function(z, mean=0, sd=1, a, b) {
  return(max.approx.frac(a/sd,b/sd,z/sd,mean/sd))
}


##############################

forwardStop <- function(pv, alpha=.10){
  if (alpha<0 || alpha>1) stop("alpha must be in [0,1]")
  if (min(pv,na.rm=T)<0 || max(pv,na.rm=T)>1) stop("pvalues must be in [0,1]")
  val=-(1/(1:length(pv)))*cumsum(log(1-pv))
  oo = which(val <= alpha)
  if (length(oo)==0) out=0
  else out = oo[length(oo)]
  return(out)
}

##############################

aicStop <- function(x, y, action, df, sigma, mult=2, ntimes=2) {
  n = length(y)
  k = length(action)
  aic = numeric(k)
  G = matrix(0,nrow=0,ncol=n)
  u = numeric(0)
  count = 0
  
  for (i in 1:k) {
    A = action[1:i]
    aic[i] = sum(lsfit(x[,A],y,intercept=F)$res^2) + mult*sigma^2*df[i]

    j = action[i]
    if (i==1) xtil = x[,j]
    else xtil = lsfit(x[,action[1:(i-1)]],x[,j],intercept=F)$res
    s = sign(sum(xtil*y))
    
    if (i==1 || aic[i] <= aic[i-1]) {
      G = rbind(G,s*xtil/sqrt(sum(xtil^2)))
      u = c(u,sqrt(mult)*sigma)
      count = 0
    }

    else {
      G = rbind(G,-s*xtil/sqrt(sum(xtil^2)))
      u = c(u,-sqrt(mult)*sigma)
      count = count+1
      if (count == ntimes) break
    }
  }

  if (i < k) {
    khat = i - ntimes
    aic = aic[1:i]
  }
  else khat = k
  
  return(list(khat=khat,G=G,u=u,aic=aic,stopped=(i<k)))
}

#these next two functions are used by the binomial and Cox options of fixedLassoInf

# Compute the truncation interval and SD of the corresponding Gaussian

TG.limits = function(Z, A, b, eta, Sigma=NULL) {

    target_estimate = sum(as.numeric(eta) * as.numeric(Z))
    

    if (max(A %*% as.numeric(Z) - b) > 0) {
        warning('Contsraint not satisfied. A %*% Z should be elementwise less than or equal to b')
    }

    if (is.null(Sigma)) {
        Sigma = diag(rep(1, n))
    }

    # compute pvalues from poly lemma:  full version from Lee et al for full matrix Sigma

    n = length(Z)
    eta = matrix(eta, ncol=1, nrow=n)
    b = as.vector(b)
    var_estimate = sum(matrix(eta, nrow=1, ncol=n) %*% (Sigma %*% matrix(eta, ncol=1, nrow=n)))
    cross_cov = Sigma %*% matrix(eta, ncol=1, nrow=n)

    resid = (diag(n) - matrix(cross_cov / var_estimate, ncol=1, nrow=n) %*% matrix(eta, nrow=1, ncol=n)) %*% Z
   
    rho = A %*% cross_cov / var_estimate
    vec = (b - as.numeric(A %*% resid)) / rho

    vlo = suppressWarnings(max(vec[rho < 0]))
    vup = suppressWarnings(min(vec[rho > 0]))

    
    sd = sqrt(var_estimate)
    return(list(vlo=vlo, vup=vup, sd=sd, estimate=target_estimate))
}

TG.pvalue = function(Z, A, b, eta, Sigma=NULL, null_value=0, bits=NULL) {

    limits.info = TG.limits(Z, A, b, eta, Sigma)

    return(TG.pvalue.base(limits.info, null_value=null_value, bits=bits))
}

TG.interval = function(Z, A, b, eta, Sigma=NULL, alpha=0.1, 
                       gridrange=c(-100,100),
                       gridpts=100, 
                       griddepth=2, 
                       flip=FALSE, 
                       bits=NULL) {

    limits.info = TG.limits(Z, A, b, eta, Sigma)

    return(TG.interval.base(limits.info, 
                            alpha=alpha, 
                            gridrange=gridrange,
                            griddepth=griddepth,
			    flip=flip,
			    bits=bits))
}

TG.interval.base = function(limits.info, alpha=0.1, 
                            gridrange=c(-100,100),
                            gridpts=100, 
                            griddepth=2, 
                            flip=FALSE, 
                            bits=NULL) {

    # compute sel intervals from poly lemmma, full version from Lee et al for full matrix Sigma

    param_grid = seq(gridrange[1] * limits.info$sd, gridrange[2] * limits.info$sd, length=gridpts)

    pivot = function(param) {
        tnorm.surv(limits.info$estimate, param, limits.info$sd, limits.info$vlo, limits.info$vup, bits) 
    }

    interval = grid.search(param_grid, pivot, alpha/2, 1-alpha/2, gridpts, griddepth)
    tailarea = c(pivot(interval[1]), 1- pivot(interval[2]))

    if (flip) {
        interval = -interval[2:1]
        tailarea = tailarea[2:1]
     }
 
     # int is not a good variable name, synonymous with integer...
     return(list(int=interval,
                 tailarea=tailarea))
}

TG.pvalue.base = function(limits.info, null_value=0, bits=NULL) {
    pv = tnorm.surv(limits.info$estimate, null_value, limits.info$sd, limits.info$vlo, limits.info$vup, bits)
    return(list(pv=pv, vlo=limits.info$vlo, vup=limits.info$vup, sd=limits.info$sd))
}


mydiag=function(x){
    if(length(x)==1) out=x
    if(length(x)>1) out=diag(x)
       return(out)
   }

    
    
# Lasso inference function (for fixed lambda). Note: here we are providing inference
# for the solution of
# min 1/2 || y - \beta_0 - X \beta ||_2^2 + \lambda || \beta ||_1

fixedLassoInf <- function(x, y, beta, lambda, family=c("gaussian","binomial","cox"),intercept=TRUE, status=NULL,
sigma=NULL, alpha=0.1,
                     type=c("partial","full"), tol.beta=1e-5, tol.kkt=0.1,
                     gridrange=c(-100,100), bits=NULL, verbose=FALSE) {

 family = match.arg(family)
  this.call = match.call()
  type = match.arg(type)

  if(family=="binomial")  {
      if(type!="partial") stop("Only type= partial allowed with binomial family")
       out=fixedLogitLassoInf(x,y,beta,lambda,alpha=alpha, type="partial", tol.beta=tol.beta, tol.kkt=tol.kkt,
                     gridrange=gridrange, bits=bits, verbose=verbose,this.call=this.call)
                      return(out)
                    }
else if(family=="cox")  {
    if(type!="partial") stop("Only type= partial allowed with Cox family")
     out=fixedCoxLassoInf(x,y,status,beta,lambda,alpha=alpha, type="partial",tol.beta=tol.beta,
          tol.kkt=tol.kkt, gridrange=gridrange, bits=bits, verbose=verbose,this.call=this.call)
                      return(out)
                    }

else{



  checkargs.xy(x,y)
  if (missing(beta) || is.null(beta)) stop("Must supply the solution beta")
  if (missing(lambda) || is.null(lambda)) stop("Must supply the tuning parameter value lambda")
  checkargs.misc(beta=beta,lambda=lambda,sigma=sigma,alpha=alpha,
                 gridrange=gridrange,tol.beta=tol.beta,tol.kkt=tol.kkt)
  if (!is.null(bits) && !requireNamespace("Rmpfr",quietly=TRUE)) {
    warning("Package Rmpfr is not installed, reverting to standard precision")
    bits = NULL
  }

  n = nrow(x)
  p = ncol(x)
  beta = as.numeric(beta)
  if (length(beta) != p) stop("Since family='gaussian', beta must have length equal to ncol(x)")

  # If glmnet was run with an intercept term, center x and y
  if (intercept==TRUE) {
    obj = standardize(x,y,TRUE,FALSE)
    x = obj$x
    y = obj$y
  }

  # Check the KKT conditions
  g = t(x)%*%(y-x%*%beta) / lambda
  if (any(abs(g) > 1+tol.kkt * sqrt(sum(y^2))))
    warning(paste("Solution beta does not satisfy the KKT conditions",
                  "(to within specified tolerances)"))

  vars = which(abs(beta) > tol.beta / sqrt(colSums(x^2)))
  if(length(vars)==0){
      cat("Empty model",fill=T)
      return()
  }
  if (any(sign(g[vars]) != sign(beta[vars])))
    warning(paste("Solution beta does not satisfy the KKT conditions",
                  "(to within specified tolerances). You might try rerunning",
                  "glmnet with a lower setting of the",
                  "'thresh' parameter, for a more accurate convergence."))

  # Get lasso polyhedral region, of form Gy >= u
  out = fixedLasso.poly(x,y,beta,lambda,vars)
  G = out$G
  u = out$u

  # Check polyhedral region
  tol.poly = 0.01
  if (min(G %*% y - u) < -tol.poly * sqrt(sum(y^2)))
    stop(paste("Polyhedral constraints not satisfied; you must recompute beta",
               "more accurately. With glmnet, make sure to use exact=TRUE in coef(),",
               "and check whether the specified value of lambda is too small",
               "(beyond the grid of values visited by glmnet).",
               "You might also try rerunning glmnet with a lower setting of the",
               "'thresh' parameter, for a more accurate convergence."))

  # Estimate sigma
  if (is.null(sigma)) {
    if (n >= 2*p) {
      oo = intercept
      sigma = sqrt(sum(lsfit(x,y,intercept=oo)$res^2)/(n-p-oo))
    }
    else {
      sigma = sd(y)
      warning(paste(sprintf("p > n/2, and sd(y) = %0.3f used as an estimate of sigma;",sigma),
                    "you may want to use the estimateSigma function"))
    }
  }

  k = length(vars)
  pv = vlo = vup = numeric(k)
  vmat = matrix(0,k,n)
  ci = tailarea = matrix(0,k,2)
  sign = numeric(k)

  if (type=="full" & p > n)
      warning(paste("type='full' does not make sense when p > n;",
                    "switching to type='partial'"))

  if (type=="partial" || p > n) {
    xa = x[,vars,drop=F]
    M = pinv(crossprod(xa)) %*% t(xa)
  }
  else {
    M = pinv(crossprod(x)) %*% t(x)
    M = M[vars,,drop=F]
  }

  for (j in 1:k) {
    if (verbose) cat(sprintf("Inference for variable %i ...\n",vars[j]))

    vj = M[j,]
    mj = sqrt(sum(vj^2))
    vj = vj / mj        # Standardize (divide by norm of vj)
    sign[j] = sign(sum(vj*y))
    vj = sign[j] * vj

    limits.info = TG.limits(y, -G, -u, vj, Sigma=diag(rep(sigma^2, n)))
    a = TG.pvalue.base(limits.info, bits=bits)
    pv[j] = a$pv
    vlo[j] = a$vlo * mj # Unstandardize (mult by norm of vj)
    vup[j] = a$vup * mj # Unstandardize (mult by norm of vj)
    vmat[j,] = vj * mj * sign[j]  # Unstandardize (mult by norm of vj)

    a = TG.interval.base(limits.info, 
                         alpha=alpha,
                         gridrange=gridrange,
			 flip=(sign[j]==-1),
                         bits=bits)
    ci[j,] = a$int * mj # Unstandardize (mult by norm of vj)
    tailarea[j,] = a$tailarea
  }

  out = list(type=type,lambda=lambda,pv=pv,ci=ci,
    tailarea=tailarea,vlo=vlo,vup=vup,vmat=vmat,y=y,
    vars=vars,sign=sign,sigma=sigma,alpha=alpha,
    sd=sigma*sqrt(rowSums(vmat^2)),
    coef0=vmat%*%y,
    call=this.call)
  class(out) = "fixedLassoInf"
  return(out)
}
}

#############################


fixedLasso.poly=
function(x, y, beta, lambda, a) {
  xa = x[,a,drop=F]
  xac = x[,!a,drop=F]
  xai = pinv(crossprod(xa))
  xap = xai %*% t(xa)
  za = sign(beta[a])
  if (length(za)>1) dz = diag(za)
  if (length(za)==1) dz = matrix(za,1,1)

#  P = diag(1,nrow(xa)) - xa %*% xap
  #NOTE: inactive constraints not needed below!

  G = -rbind(
   #   1/lambda * t(xac) %*% P,
   # -1/lambda * t(xac) %*% P,
    -dz %*% xap
      )
     lambda2=lambda
     if(length(lambda)>1) lambda2=lambda[a]
  u = -c(
   #   1 - t(xac) %*% t(xap) %*% za,
   #   1 + t(xac) %*% t(xap) %*% za,
    -lambda2 * dz %*% xai %*% za)

  return(list(G=G,u=u))
}

##############################

print.fixedLassoInf <- function(x, tailarea=TRUE, ...) {
  cat("\nCall:\n")
  dput(x$call)

  cat(sprintf("\nStandard deviation of noise (specified or estimated) sigma = %0.3f\n",
              x$sigma))

  cat(sprintf("\nTesting results at lambda = %0.3f, with alpha = %0.3f\n",x$lambda,x$alpha))
  cat("",fill=T)
  tab = cbind(x$vars,
    round(x$coef0,3),
    round(x$coef0 / x$sd,3),
    round(x$pv,3),round(x$ci,3))
  colnames(tab) = c("Var", "Coef", "Z-score", "P-value", "LowConfPt", "UpConfPt")
  if (tailarea) {
    tab = cbind(tab,round(x$tailarea,3))
    colnames(tab)[(ncol(tab)-1):ncol(tab)] = c("LowTailArea","UpTailArea")
  }
  rownames(tab) = rep("",nrow(tab))
  print(tab)

  cat(sprintf("\nNote: coefficients shown are %s regression coefficients\n",
              ifelse(x$type=="partial","partial","full")))
  invisible()
}

#estimateLambda <- function(x, sigma, nsamp=1000){
#  checkargs.xy(x,rep(0,nrow(x)))
#  if(nsamp < 10) stop("More Monte Carlo samples required for estimation")
#  if (length(sigma)!=1) stop("sigma should be a number > 0")
 # if (sigma<=0) stop("sigma should be a number > 0")

 # n = nrow(x)
 # eps = sigma*matrix(rnorm(nsamp*n),n,nsamp)
 # lambda = 2*mean(apply(t(x)%*%eps,2,max))
 # return(lambda)
#}


#estimateLambda <- function(x, sigma, nsamp=1000){
#  checkargs.xy(x,rep(0,nrow(x)))
#  if(nsamp < 10) stop("More Monte Carlo samples required for estimation")
#  if (length(sigma)!=1) stop("sigma should be a number > 0")
 # if (sigma<=0) stop("sigma should be a number > 0")

 # n = nrow(x)
 # eps = sigma*matrix(rnorm(nsamp*n),n,nsamp)
 # lambda = 2*mean(apply(t(x)%*%eps,2,max))
 # return(lambda)
#}





#out = fixedLassoInf(x,y,w,l1,sigma=sigma,alpha=0.05)
#out

time: 24.1 ms


time: 939 ms
